# Train

> Pew pew pew

In [ ]:
#| default_exp train

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import torch, torch.nn as nn, lightning.pytorch as pl

from torch.optim.lr_scheduler import OneCycleLR, CosineAnnealingWarmRestarts

from pftsleep.transformers import PatchTFTSimple
from pftsleep.loss import masked_mae_loss, cosine_similarity_loss, masked_mse_loss, FocalLoss, KLDivLoss, huber_loss, patch_continuity_loss

## Self Supervised PatchTFT Lightning

In [ ]:
#| export
class PatchTFTSimpleLightning(pl.LightningModule):
    def __init__(self,
                 learning_rate,
                 train_size,
                 batch_size,
                 channels,
                 metrics,
                 precalculate_onebatch_stft_stats=False,
                 use_sequence_padding_mask=False,
                 loss_func='mse',
                 max_lr=0.01,
                 weight_decay=0.,
                 epochs=100,
                 one_cycle_scheduler=True,
                 optimizer_type='Adam',
                 scheduler_type='OneCycle',
                 cross_attention=False, # not implemented
                 use_mask=False,
                 patch_continuity_loss=0, # indicator and ratio of patch continuity loss function, which examines ensures patches dont have large discontinuities
                 huber_delta=None, # huber loss delta, not used otherwise
                 **patchmeup_kwargs
                 ):
        super().__init__()
        assert loss_func.lower() in ['mae','mse','cosine','huber']
        self.scheduler_type = scheduler_type
        if self.scheduler_type is not None:
            assert self.scheduler_type.lower() in ['onecycle', 'cosineannealingwarmrestarts'], "scheduler must be either OneCycle, CosineAnnealingWarmRestarts, or None"
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        self.max_lr = max_lr
        self.train_size = train_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.metrics = metrics
        self.one_cycle_scheduler = one_cycle_scheduler
        self.channels = channels
        self.use_mask = use_mask
        self.loss_func = loss_func
        self.huber_delta = huber_delta
        self.patch_continuity_loss = patch_continuity_loss
        self.precalculate_onebatch_stft_stats = precalculate_onebatch_stft_stats
        self.use_sequence_padding_mask = use_sequence_padding_mask
        self.cross_attention = cross_attention
        self.optimizer_type = optimizer_type
        self.weight_decay = weight_decay
        self.model = PatchTFTSimple(**patchmeup_kwargs)
    
    def forward(self, x, sequence_padding_mask=None):
        if self.training:
            x_hat, y, training_mask, key_padding_mask = self.model(x, sequence_padding_mask=sequence_padding_mask)
            return x_hat, y, training_mask, key_padding_mask
        else:
            z, x_hat, y, training_mask, key_padding_mask = self.model(x, sequence_padding_mask=sequence_padding_mask)
            return z, x_hat, y, training_mask, key_padding_mask

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        if self.use_sequence_padding_mask:
            x, _, sequence_padding_mask = batch
        else:
            x, _ = batch
            sequence_padding_mask = None
        x_hat, y, training_mask, key_padding_mask = self.model(x, sequence_padding_mask=sequence_padding_mask)
        if self.patch_continuity_loss > 0:
            patch_cont_loss = patch_continuity_loss(x_hat) * self.patch_continuity_loss
        else:
            patch_cont_loss = 0
        if self.loss_func == 'cosine':
            loss = cosine_similarity_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        elif self.loss_func in ['MAE','mae']:
            loss = masked_mae_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        elif self.loss_func in ['Huber','huber']:
            loss = huber_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask, delta=self.huber_delta if self.huber_delta is not None else 1)
        else:
            loss = masked_mse_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        loss = loss + patch_cont_loss
        loss = loss.to(self.device)
        self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        if self.patch_continuity_loss > 0:
            self.log('patch_continuity_loss', patch_cont_loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log_dict({f'train_{metric.__name__}':metric(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask) for metric in self.metrics}, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        if self.use_sequence_padding_mask:
            x, _, sequence_padding_mask = batch
        else:
            x, _ = batch
            sequence_padding_mask = None
        _, x_hat, y, training_mask, key_padding_mask = self.model(x, sequence_padding_mask=sequence_padding_mask)
        if self.patch_continuity_loss > 0:
            patch_cont_loss = patch_continuity_loss(x_hat) * self.patch_continuity_loss
        else:
            patch_cont_loss = 0
        if self.loss_func == 'cosine':
            loss = cosine_similarity_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        elif self.loss_func in ['MAE','mae']:
            loss = masked_mae_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        elif self.loss_func in ['Huber','huber']:
            loss = huber_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask, delta=self.huber_delta if self.huber_delta is not None else 1)
        else:
            loss = masked_mse_loss(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask)
        loss = loss + patch_cont_loss
        loss = loss.to(self.device)
        self.log("val_loss", loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        if self.patch_continuity_loss > 0:
            self.log('patch_continuity_loss', patch_cont_loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        self.log_dict({f'valid_{metric.__name__}':metric(x_hat, y, training_mask, use_mask=self.use_mask, padding_mask=key_padding_mask) for metric in self.metrics}, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay) if self.optimizer_type.lower() == 'adamw' else\
                     torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        if self.scheduler_type.lower() == 'onecycle':
            scheduler = OneCycleLR(optimizer, max_lr=self.max_lr, epochs=self.epochs, steps_per_epoch=(self.train_size//self.batch_size))
            lr_scheduler = {'scheduler': scheduler, 'interval': 'step'}
            return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
        elif self.scheduler_type.lower() == 'cosineannealingwarmrestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=self.epochs//10, T_mult=2, eta_min=1e-8) # lr max is initial LR
            lr_scheduler = {'scheduler': scheduler, 'interval': 'epoch'}
            return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
        else:
            return optimizer

In [ ]:
#| notest
class EncoderTestFreezingWeights(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Linear(750,512)
    def forward(self, x):
        x = self.encoder(x)
        return x

class DecoderTest(pl.LightningModule):
    def __init__(self, encoder):
        super().__init__()
        self.encoder = encoder
        self.encoder.freeze()
    def training_step(self, batch, batch_idx):
        print(list(self.encoder.parameters()))
        print(self.encoder.training)
        # x = self.encoder(x)
        # return x

encoder = EncoderTestFreezingWeights()
decoder = DecoderTest(encoder)

decoder.training_step(0,0)

## Supervised Training with Linear Probing

### PatchTFT Masked Autoregression SS Prediction

In [ ]:
#| export
class PatchTFTSleepStage(pl.LightningModule):
    def __init__(self, 
                 learning_rate, # desired learning rate, initial learning rate in if one_cycle_scheduler
                 train_size, # the training data size (for one_cycle_scheduler=True)
                 batch_size, # the batch size (for one_cycle_scheduler=True)
                 linear_probing_head, # model head to train
                 metrics=[], # metrics to calculate
                 fine_tune=False, # indicator to finetune encoder model or perform linear probing and freeze encoder weights
                 loss_fxn='CrossEntropy', # loss function to use, can be CrossEntropy or FocalLoss
                 class_weights=None, # weights of classes to use in CE loss fxn
                 gamma=2., # for focal loss
                 label_smoothing=0, # label smoothing for cross entropy loss
                 use_sequence_padding_mask=False, #indicator to use the sequence padding mask when training/in the loss fxn
                 y_padding_mask=-100, # padded value that was added to target and indice to ignore when computing loss
                 max_lr=0.01, # maximum learning rate for one_cycle_scheduler
                 epochs=100, # number of epochs for one_cycle_scheduler
                 one_cycle_scheduler=True, # indicator to use a one cycle scheduler to vary the learning rate 
                 weight_decay=0., # weight decay for Adam optimizer
                 pretrained_encoder_path=None, # path of the pretrained model to use for linear probing
                 optimizer_type='Adam', # optimizer to use, 'Adam' or 'AdamW'
                 scheduler_type='OneCycle', # scheduler to use, 'OneCycle' or 'CosineAnnealingWarmRestarts'
                 preloaded_model=None, # loaded pretrained model to use for linear probing
                 torch_model_name='model', # name of the pytorch model within the lightning model module, this is to remove layers (for example lightning_model.pytorch_model.head = nn.Identity())
                 remove_pretrain_layers=['head', 'mask'], # layers within the lightning model or lightning model.pytorch_model to remove
                 return_softmax=True, # indicator to return softmax probabilities in forward and predict_step
                 ):
        super().__init__()
        self.encoder = preloaded_model
        assert loss_fxn.lower() in ['crossentropy', 'focalloss', 'kldivloss'], "loss_fxn must be either CrossEntropy or FocalLoss or KLDivLoss"
        if remove_pretrain_layers is not None and len(remove_pretrain_layers) > 0:
            for l in remove_pretrain_layers:
                if torch_model_name is not None:
                    setattr(getattr(self.encoder, torch_model_name), l, nn.Identity())
                else:
                    setattr(self.encoder, l, nn.Identity())
        self.scheduler_type = scheduler_type
        if self.scheduler_type is not None:
            assert self.scheduler_type.lower() in ['onecycle', 'cosineannealingwarmrestarts'], "scheduler must be either OneCycle, CosineAnnealingWarmRestarts, or None"
        self.one_cycle_scheduler = one_cycle_scheduler
        self.weight_decay = weight_decay
        self.label_smoothing = label_smoothing
        self.learning_rate = learning_rate
        self.max_lr = max_lr
        self.epochs = epochs
        self.train_size = train_size
        self.batch_size = batch_size
        self.metrics = metrics
        self.y_padding_mask = y_padding_mask
        self.class_weights = class_weights
        self.use_sequence_padding_mask = use_sequence_padding_mask
        self.loss_fxn = loss_fxn
        self.gamma = gamma
        self.fine_tune = fine_tune
        self.return_softmax = return_softmax
        if not self.fine_tune:
            self.encoder.freeze() # freeze the encoder weights
            #self.encoder.eval() # should this be on here?
        self.feedforward = linear_probing_head #RNNProbingHead #TimeDistributedConvolutionalFeedForward(**linear_probing_kwargs)#LinearProbingHead(**linear_probing_kwargs)
        #self.feedforward = self.feedforward.to(self.device)
        self.optimizer_type = optimizer_type
        self.save_hyperparameters(ignore=['linear_probing_head', 'preloaded_model'])

    def forward(self, x, sequence_padding_mask=None):
        """
        In: [bs, n_channels, seq_len]
        """
        x = self.encoder(x, sequence_padding_mask=sequence_padding_mask) # [bs, n_channels, d_model, n_ffts/n_patches]
        if isinstance(x, tuple):
            x = x[0]
        x = self.feedforward(x, return_softmax=self.return_softmax, sequence_padding_mask=sequence_padding_mask) # [bs, n_classes, pred_len_seconds]
        return x
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        if self.use_sequence_padding_mask:
            x, _, sequence_padding_mask = batch
        else:
            x,_ = batch
            sequence_padding_mask = None
        x = self.encoder(x, sequence_padding_mask = sequence_padding_mask)
        if isinstance(x, tuple):
            x = x[0]
        preds = self.feedforward(x, return_softmax=self.return_softmax, sequence_padding_mask=sequence_padding_mask)
        return preds
        
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        if self.use_sequence_padding_mask:
            x, y, sequence_padding_mask = batch
        else:
            x, y = batch
            sequence_padding_mask = None
        if not self.fine_tune:
            self.encoder.eval()
            with torch.no_grad():
                x = self.encoder(x, sequence_padding_mask=sequence_padding_mask)
        else:
            x = self.encoder(x, sequence_padding_mask=sequence_padding_mask)
        if isinstance(x, tuple):
            x = x[0]
        x = self.feedforward(x, sequence_padding_mask=sequence_padding_mask)
        if self.loss_fxn.lower() == 'kldivloss':
            loss = KLDivLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, ignore_index=self.y_padding_mask)(x,y)
        else:
            ce_loss = nn.CrossEntropyLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, label_smoothing=self.label_smoothing, ignore_index=self.y_padding_mask)
            focal_loss = FocalLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, gamma=self.gamma, ignore_index=self.y_padding_mask)
            loss = ce_loss(x,y) if self.loss_fxn == 'CrossEntropy' else focal_loss(x,y)
            self.log('train_ce_loss' if self.loss_fxn != 'CrossEntropy' else 'train_focal_loss', ce_loss(x,y) if self.loss_fxn != 'CrossEntropy' else focal_loss(x,y), prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
            self.log_dict({f'train_{metric.__name__}':metric(x, y) for metric in self.metrics}, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)
        #loss = loss.to(x.device)
        self.log('train_loss', loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        if self.use_sequence_padding_mask:
            x, y, sequence_padding_mask = batch
        else:
            x, y = batch
            sequence_padding_mask = None
        x = self.encoder(x, sequence_padding_mask=sequence_padding_mask) # val should be in eval mode
        if isinstance(x, tuple):
            x = x[0]
        x = self.feedforward(x, sequence_padding_mask=sequence_padding_mask)
        if self.loss_fxn.lower() == 'kldivloss':
            loss = KLDivLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, ignore_index=self.y_padding_mask)(x,y)
        else:
            ce_loss = nn.CrossEntropyLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, label_smoothing=self.label_smoothing, ignore_index=self.y_padding_mask)
            focal_loss = FocalLoss(weight=self.class_weights.to(x.device) if self.class_weights is not None else None, gamma=self.gamma, ignore_index=self.y_padding_mask)
            loss = ce_loss(x,y) if self.loss_fxn == 'CrossEntropy' else focal_loss(x,y)
            self.log('val_ce_loss' if self.loss_fxn != 'CrossEntropy' else 'val_focal_loss', ce_loss(x,y) if self.loss_fxn != 'CrossEntropy' else focal_loss(x,y), prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)
            self.log_dict({f'val_{metric.__name__}':metric(x, y) for metric in self.metrics}, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True)
        #loss = loss.to(x.device)
        self.log('val_loss', loss, prog_bar=True, on_step=True, on_epoch=True, sync_dist=True)

    def on_validation_epoch_start(self):
        torch.cuda.empty_cache()
        
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay) if self.optimizer_type.lower() == 'adamw' else\
                     torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        if self.scheduler_type.lower() == 'onecycle':
            scheduler = OneCycleLR(optimizer, max_lr=self.max_lr, epochs=self.epochs, steps_per_epoch=(self.train_size//self.batch_size))
            lr_scheduler = {'scheduler': scheduler, 'interval': 'step'}
            return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
        elif self.scheduler_type.lower() == 'cosineannealingwarmrestarts':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=self.epochs//10, T_mult=2, eta_min=1e-8)
            lr_scheduler = {'scheduler': scheduler, 'interval': 'epoch'}
            return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
        else:
            return optimizer

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()